<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Spark-MLlib-Tuning" data-toc-modified-id="Spark-MLlib-Tuning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-tuning.html" target="_blank">Spark MLlib Tuning</a></a></span></li><li><span><a href="#Hyperopt" data-toc-modified-id="Hyperopt-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><a href="https://github.com/hyperopt/hyperopt" target="_blank">Hyperopt</a></a></span><ul class="toc-item"><li><span><a href="#XGBoost-Tuning" data-toc-modified-id="XGBoost-Tuning-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/" target="_blank">XGBoost Tuning</a></a></span><ul class="toc-item"><li><span><a href="#Objective-function" data-toc-modified-id="Objective-function-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Objective function</a></span></li><li><span><a href="#Tune-number-of-trees" data-toc-modified-id="Tune-number-of-trees-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Tune number of trees</a></span></li><li><span><a href="#Tune-tree-specific-parameters" data-toc-modified-id="Tune-tree-specific-parameters-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Tune tree-specific parameters</a></span><ul class="toc-item"><li><span><a href="#Tune-max_depth,-min_child_weight" data-toc-modified-id="Tune-max_depth,-min_child_weight-2.1.3.1"><span class="toc-item-num">2.1.3.1&nbsp;&nbsp;</span>Tune max_depth, min_child_weight</a></span></li><li><span><a href="#Tune-gamma" data-toc-modified-id="Tune-gamma-2.1.3.2"><span class="toc-item-num">2.1.3.2&nbsp;&nbsp;</span>Tune gamma</a></span></li><li><span><a href="#Tune-subsample,-colsample_bytree" data-toc-modified-id="Tune-subsample,-colsample_bytree-2.1.3.3"><span class="toc-item-num">2.1.3.3&nbsp;&nbsp;</span>Tune subsample, colsample_bytree</a></span></li></ul></li><li><span><a href="#Tune-regularization-parameters" data-toc-modified-id="Tune-regularization-parameters-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Tune regularization parameters</a></span></li><li><span><a href="#Lower-the-learning-rate-and-decide-the-optimal-parameters" data-toc-modified-id="Lower-the-learning-rate-and-decide-the-optimal-parameters-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Lower the learning rate and decide the optimal parameters</a></span></li></ul></li><li><span><a href="#LogisticRegression-Tuning" data-toc-modified-id="LogisticRegression-Tuning-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LogisticRegression Tuning</a></span></li><li><span><a href="#Optional-MongoTrials" data-toc-modified-id="Optional-MongoTrials-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Optional <a href="https://hyperopt.github.io/hyperopt/scaleout/mongodb/" target="_blank">MongoTrials</a></a></span><ul class="toc-item"><li><span><a href="#XGBoost-Tuning" data-toc-modified-id="XGBoost-Tuning-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>XGBoost Tuning</a></span></li></ul></li></ul></li><li><span><a href="#Results" data-toc-modified-id="Results-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Results</a></span></li></ul></div>

Продолжаем работать над задачей CTR-prediction с использованием датасета от Criteo.

Описание задачи и данных можно посмотреть в notebook'e предыдущей практики (`sgd_logreg_nn/notebooks/ctr_prediction_mllib.ipynb`).

In [1]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row

COMMON_PATH = '/workspace/common'

sys.path.append(os.path.join(COMMON_PATH, 'utils'))

os.environ['PYSPARK_SUBMIT_ARGS'] = """
--jars {common}/xgboost4j-spark-0.72.jar,{common}/xgboost4j-0.72.jar
--py-files {common}/sparkxgb.zip pyspark-shell
""".format(common=COMMON_PATH).replace('\n', ' ')

spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "6g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

from metrics import rocauc, logloss, ne, get_ate
from processing import split_by_col

from sparkxgb.xgboost import *

In [2]:
DATA_PATH = '/workspace/data/criteo'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')

In [3]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

**Remark** Необязательно использовать половину датасета и всего две категориальные переменные. Можно использовать больше данных, если вам позволяет ваша конфигурация

In [4]:
df = df.sample(False, 0.5)

In [5]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [6]:
df = df.fillna(0, subset=num_columns)

In [7]:
from pyspark.ml import PipelineModel


pipeline_model = PipelineModel.load(os.path.join(DATA_PATH, 'pipeline_model'))

In [8]:
df = pipeline_model \
    .transform(df) \
    .select(F.col('_c0').alias('label'), 'features', 'id') \
    .cache()

df.count()

1833342

In [9]:
train_df, val_df, test_df = split_by_col(df, 'id', [0.8, 0.1, 0.1])

# [Spark MLlib Tuning](https://spark.apache.org/docs/latest/ml-tuning.html)

У имеющегося в Spark'e метода HPO есть два существенных недостатка, которые делают его мало пригодным в контексте нашей задачи:

1. `ParamGridBuilder` - поиск по сетке
2. `TrainValidationSplit` - делит данные случайнм образом

# [Hyperopt](https://github.com/hyperopt/hyperopt)

Установим `hyperopt`

In [10]:
!pip3.5 install hyperopt

     |████████████████████████████████| 1.9 MB 520 kB/s eta 0:00:01
     |████████████████████████████████| 829 kB 458 kB/s eta 0:00:01
     |████████████████████████████████| 59 kB 630 kB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 763 kB/s eta 0:00:01
    Running setup.py install for future ... done
    Running setup.py install for networkx ... done


## [XGBoost Tuning](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/)

> [Notes on Parameter Tuning](https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html)

### Objective function

In [11]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import scipy.stats as st


def objective(space):
    estimator = XGBoostEstimator(**space)
    print('SPACE:', estimator._input_kwargs_processed())
    success = False
    attempts = 0
    model = None
    while not success and attempts < 2:
        try:
            model = estimator.fit(train_df)
            success = True
        except Exception as e:
            attempts += 1
            print(e)
            print('Try again')
        
    log_loss = logloss(model, val_df, probabilities_col='probabilities')
    roc_auc = rocauc(model, val_df, probabilities_col='probabilities')
    
    print('LOG-LOSS: {}, ROC-AUC: {}'.format(log_loss, roc_auc))

    return {'loss': log_loss, 'rocauc': roc_auc, 'status': STATUS_OK }

In [40]:
static_params = {
    'featuresCol': "features", 
    'labelCol': "label", 
    'predictionCol': "prediction",
    'eval_metric': 'logloss',
    'objective': 'binary:logistic',
    'nthread': 1,
    'silent': 0,
    'nworkers': 4
}

Fix baseline parameters and train baseline model

In [13]:
CONTROL_NAME = 'xgb baseline'

baseline_params = {
    'colsample_bytree': 0.9,
    'eta': 0.15,
    'gamma': 0.9,
    'max_depth': 6,
    'min_child_weight': 50.0,
    'subsample': 0.9,
    'num_round': 20
}

baseline_model = XGBoostEstimator(**{**static_params, **baseline_params}).fit(train_df)

In [14]:
baseline_rocauc = rocauc(baseline_model, val_df, probabilities_col='probabilities')
baseline_rocauc

0.726822630845867

In [15]:
all_metrics = {}

In [16]:
baseline_test_metrics = {
    'logloss': logloss(baseline_model, test_df, probabilities_col='probabilities'),
    'rocauc': rocauc(baseline_model, test_df, probabilities_col='probabilities')
}

all_metrics[CONTROL_NAME] = baseline_test_metrics

### Tune number of trees

> Choose a relatively high learning rate. Generally a learning rate of 0.1 works but somewhere between 0.05 to 0.3 should work for different problems. Determine the optimum number of trees for this learning rate.

In [29]:
%%time

num_round_choice = [10, 20, 40, 100]
eta_choice = [0.5, 0.10, 0.15, 0.20, 0.30]

space = {
    # Optimize
    'num_round': hp.choice('num_round', num_round_choice),
    'eta': hp.choice('eta', eta_choice),
    
    # Fixed    
    'max_depth': baseline_params['max_depth'],
    'min_child_weight': baseline_params['min_child_weight'],
    'subsample': baseline_params['subsample'],
    'gamma': baseline_params['gamma'],
    'colsample_bytree': baseline_params['colsample_bytree'],
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

SPACE:                                                
{'nthread': 1, 'subsample': 0.9, 'max_depth': 6, 'num_round': 10, 'min_child_weight': 50.0, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'objective': 'binary:logistic', 'nworkers': 1, 'predictionCol': 'prediction', 'silent': 0, 'eta': 0.15, 'gamma': 0.9, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5233571687421811, ROC-AUC: 0.7218132639750544
SPACE:                                                                          
{'nthread': 1, 'subsample': 0.9, 'max_depth': 6, 'num_round': 100, 'min_child_weight': 50.0, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'objective': 'binary:logistic', 'nworkers': 1, 'predictionCol': 'prediction', 'silent': 0, 'eta': 0.1, 'gamma': 0.9, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5021889577229195, ROC-AUC: 0.7349656930050602                       
SPACE:                                                                          
{'nthread': 1, 'subsample'

LOG-LOSS: 0.5007997276627983, ROC-AUC: 0.7367195013338069                         
SPACE:                                                                             
{'nthread': 1, 'subsample': 0.9, 'max_depth': 6, 'num_round': 10, 'min_child_weight': 50.0, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'objective': 'binary:logistic', 'nworkers': 1, 'predictionCol': 'prediction', 'silent': 0, 'eta': 0.15, 'gamma': 0.9, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5233571687421811, ROC-AUC: 0.7218132639750525                          
SPACE:                                                                             
{'nthread': 1, 'subsample': 0.9, 'max_depth': 6, 'num_round': 40, 'min_child_weight': 50.0, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'objective': 'binary:logistic', 'nworkers': 1, 'predictionCol': 'prediction', 'silent': 0, 'eta': 0.1, 'gamma': 0.9, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5062206416669133, ROC-AUC: 0.7293529

In [30]:
best

{'eta': 0, 'num_round': 3}

Обратите внимание на то, что в случае с `hp.choice` в переменной `best` хранится не конкретное значение гиперпараметра, а его индекс из списка, например, `num_round_choice`

In [21]:
eta = eta_choice[best['eta']]  # change me!
num_round = num_round_choice[best['num_round']]  # change me!

### Tune tree-specific parameters

> Tune tree-specific parameters ( max_depth, min_child_weight, gamma, subsample, colsample_bytree) for decided learning rate and number of trees. Note that we can choose different parameters to define a tree and I’ll take up an example here.

#### Tune max_depth, min_child_weight

In [41]:
%%time

max_depth_choice = [4, 6, 8, 12]
min_child_weight_choice = [1., 10., 50., 100., ]

space = {
    # Optimize
    'max_depth': hp.choice('max_depth', max_depth_choice),
    'min_child_weight': hp.choice('min_child_weight', min_child_weight_choice),
    
    # Fixed    
    'num_round': baseline_params['num_round'],
    'eta': baseline_params['eta'],
    'subsample': baseline_params['subsample'],
    'gamma': baseline_params['gamma'],
    'colsample_bytree': baseline_params['colsample_bytree'],
    
    **static_params
}


trials_depth_child_weight = Trials()
best_depth_child_weight = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials_depth_child_weight)

SPACE:                                                
{'nthread': 1, 'subsample': 0.9, 'max_depth': 8, 'num_round': 20, 'min_child_weight': 10.0, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'objective': 'binary:logistic', 'nworkers': 4, 'predictionCol': 'prediction', 'silent': 0, 'eta': 0.15, 'gamma': 0.9, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5058900708109083, ROC-AUC: 0.7302854871032325
SPACE:                                                                          
{'nthread': 1, 'subsample': 0.9, 'max_depth': 8, 'num_round': 20, 'min_child_weight': 100.0, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'objective': 'binary:logistic', 'nworkers': 4, 'predictionCol': 'prediction', 'silent': 0, 'eta': 0.15, 'gamma': 0.9, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5062720422223786, ROC-AUC: 0.7301174964126633                       
SPACE:                                                                          
{'nthread': 1, 'subsample

LOG-LOSS: 0.50627389267702, ROC-AUC: 0.7300623775922761                          
SPACE:                                                                           
{'nthread': 1, 'subsample': 0.9, 'max_depth': 8, 'num_round': 20, 'min_child_weight': 100.0, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'objective': 'binary:logistic', 'nworkers': 4, 'predictionCol': 'prediction', 'silent': 0, 'eta': 0.15, 'gamma': 0.9, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.50627389267702, ROC-AUC: 0.7300623775922758                          
SPACE:                                                                           
{'nthread': 1, 'subsample': 0.9, 'max_depth': 6, 'num_round': 20, 'min_child_weight': 100.0, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'objective': 'binary:logistic', 'nworkers': 4, 'predictionCol': 'prediction', 'silent': 0, 'eta': 0.15, 'gamma': 0.9, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5085099216111754, ROC-AUC: 0.72709513008

In [42]:
best_depth_child_weight

{'max_depth': 3, 'min_child_weight': 1}

#### Tune gamma

In [44]:
%%time

gamma_choice = [0., 0.1, 0.3, 0.9, 3., 10.]

space = {
    # Optimize
    'gamma': hp.choice('gamma', gamma_choice),
    
    # Fixed    
    'num_round': baseline_params['num_round'],
    'eta': baseline_params['eta'],
    'max_depth': baseline_params['max_depth'],
    'min_child_weight': baseline_params['min_child_weight'],
    'subsample': baseline_params['subsample'],
    'colsample_bytree': baseline_params['colsample_bytree'],
    
    **static_params
}


trials_gamma = Trials()
best_gamma = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials_gamma)

SPACE:                                                
{'nthread': 1, 'subsample': 0.9, 'max_depth': 6, 'num_round': 20, 'min_child_weight': 50.0, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'objective': 'binary:logistic', 'nworkers': 4, 'predictionCol': 'prediction', 'silent': 0, 'eta': 0.15, 'gamma': 0.9, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5085891294460596, ROC-AUC: 0.7268289606994125
SPACE:                                                                          
{'nthread': 1, 'subsample': 0.9, 'max_depth': 6, 'num_round': 20, 'min_child_weight': 50.0, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'objective': 'binary:logistic', 'nworkers': 4, 'predictionCol': 'prediction', 'silent': 0, 'eta': 0.15, 'gamma': 3.0, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5085244672407123, ROC-AUC: 0.72700704067199                         
SPACE:                                                                          
{'nthread': 1, 'subsample'

LOG-LOSS: 0.5084969906871213, ROC-AUC: 0.7270795848028542                        
SPACE:                                                                           
{'nthread': 1, 'subsample': 0.9, 'max_depth': 6, 'num_round': 20, 'min_child_weight': 50.0, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'objective': 'binary:logistic', 'nworkers': 4, 'predictionCol': 'prediction', 'silent': 0, 'eta': 0.15, 'gamma': 10.0, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5086202720225981, ROC-AUC: 0.7270201258959514                        
SPACE:                                                                           
{'nthread': 1, 'subsample': 0.9, 'max_depth': 6, 'num_round': 20, 'min_child_weight': 50.0, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'objective': 'binary:logistic', 'nworkers': 4, 'predictionCol': 'prediction', 'silent': 0, 'eta': 0.15, 'gamma': 0.0, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5086348508565168, ROC-AUC: 0.727027648774

In [45]:
best_gamma

{'gamma': 4}

#### Tune subsample, colsample_bytree

In [50]:
%%time

subsample_choice = [.2, .5, .8, 1.]
colsample_bytree_choice = [.2, .5, .8, 1.]

space = {
    # Optimize
    'subsample': hp.choice('subsample', subsample_choice),
    'colsample_bytree': hp.choice('colsample_bytree', colsample_bytree_choice),
    
    # Fixed    
    'num_round': baseline_params['num_round'],
    'eta': baseline_params['eta'],
    'max_depth': baseline_params['max_depth'],
    'min_child_weight': baseline_params['min_child_weight'],
    'gamma': baseline_params['gamma'],
    
    **static_params
}


trials_sample = Trials()
best_sample = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials_sample)

SPACE:                                                
{'nthread': 1, 'subsample': 0.8, 'max_depth': 6, 'num_round': 20, 'min_child_weight': 50.0, 'colsample_bytree': 0.2, 'featuresCol': 'features', 'objective': 'binary:logistic', 'nworkers': 4, 'predictionCol': 'prediction', 'silent': 0, 'eta': 0.15, 'gamma': 0.9, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5160354565958816, ROC-AUC: 0.721767719361712
SPACE:                                                                          
{'nthread': 1, 'subsample': 0.2, 'max_depth': 6, 'num_round': 20, 'min_child_weight': 50.0, 'colsample_bytree': 0.5, 'featuresCol': 'features', 'objective': 'binary:logistic', 'nworkers': 4, 'predictionCol': 'prediction', 'silent': 0, 'eta': 0.15, 'gamma': 0.9, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5097006338364162, ROC-AUC: 0.7259529420231307                       
SPACE:                                                                          
{'nthread': 1, 'subsample':

LOG-LOSS: 0.5093938333272552, ROC-AUC: 0.7261365071572602                        
SPACE:                                                                           
{'nthread': 1, 'subsample': 1.0, 'max_depth': 6, 'num_round': 20, 'min_child_weight': 50.0, 'colsample_bytree': 0.8, 'featuresCol': 'features', 'objective': 'binary:logistic', 'nworkers': 4, 'predictionCol': 'prediction', 'silent': 0, 'eta': 0.15, 'gamma': 0.9, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5086945133229195, ROC-AUC: 0.7268144996220817                        
SPACE:                                                                           
{'nthread': 1, 'subsample': 0.5, 'max_depth': 6, 'num_round': 20, 'min_child_weight': 50.0, 'colsample_bytree': 0.8, 'featuresCol': 'features', 'objective': 'binary:logistic', 'nworkers': 4, 'predictionCol': 'prediction', 'silent': 0, 'eta': 0.15, 'gamma': 0.9, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.508725493093763, ROC-AUC: 0.72680351328987

In [51]:
best_sample

{'colsample_bytree': 3, 'subsample': 3}

### Tune regularization parameters

> Tune regularization parameters (lambda, alpha) for xgboost which can help reduce model complexity and enhance performance.

In [54]:
%%time

alpha_choice = [0., 0.1, 0.5, 1., 10.]
lambda_choice = [0., 0.1, 0.5, 1., 10.]

space = {
    # Optimize
    'alpha': hp.choice('alpha', alpha_choice),
    'reg_lambda': hp.choice('lambda', lambda_choice),
    
    # Fixed    
    'num_round': baseline_params['num_round'],
    'eta': baseline_params['eta'],
    'max_depth': baseline_params['max_depth'],
    'min_child_weight': baseline_params['min_child_weight'],
    'subsample': baseline_params['subsample'],
    'gamma': baseline_params['gamma'],
    'colsample_bytree': baseline_params['colsample_bytree'],
    
    **static_params
}


trials_reg = Trials()
best_reg = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials_reg)

SPACE:                                                
{'nthread': 1, 'subsample': 0.9, 'eval_metric': 'logloss', 'featuresCol': 'features', 'num_round': 20, 'min_child_weight': 50.0, 'lambda': 10.0, 'colsample_bytree': 0.9, 'alpha': 0.5, 'objective': 'binary:logistic', 'nworkers': 4, 'predictionCol': 'prediction', 'silent': 0, 'eta': 0.15, 'gamma': 0.9, 'max_depth': 6, 'labelCol': 'label'}
LOG-LOSS: 0.5086635632729125, ROC-AUC: 0.726921885712003
SPACE:                                                                          
{'nthread': 1, 'subsample': 0.9, 'eval_metric': 'logloss', 'featuresCol': 'features', 'num_round': 20, 'min_child_weight': 50.0, 'lambda': 1.0, 'colsample_bytree': 0.9, 'alpha': 10.0, 'objective': 'binary:logistic', 'nworkers': 4, 'predictionCol': 'prediction', 'silent': 0, 'eta': 0.15, 'gamma': 0.9, 'max_depth': 6, 'labelCol': 'label'}
LOG-LOSS: 0.5087141799029173, ROC-AUC: 0.7266753415280959                       
SPACE:                                          

LOG-LOSS: 0.5084905324758686, ROC-AUC: 0.7269608578606717                        
SPACE:                                                                           
{'nthread': 1, 'subsample': 0.9, 'eval_metric': 'logloss', 'featuresCol': 'features', 'num_round': 20, 'min_child_weight': 50.0, 'lambda': 1.0, 'colsample_bytree': 0.9, 'alpha': 0.1, 'objective': 'binary:logistic', 'nworkers': 4, 'predictionCol': 'prediction', 'silent': 0, 'eta': 0.15, 'gamma': 0.9, 'max_depth': 6, 'labelCol': 'label'}
LOG-LOSS: 0.5085397813957346, ROC-AUC: 0.7270032856518374                        
SPACE:                                                                           
{'nthread': 1, 'subsample': 0.9, 'eval_metric': 'logloss', 'featuresCol': 'features', 'num_round': 20, 'min_child_weight': 50.0, 'lambda': 1.0, 'colsample_bytree': 0.9, 'alpha': 10.0, 'objective': 'binary:logistic', 'nworkers': 4, 'predictionCol': 'prediction', 'silent': 0, 'eta': 0.15, 'gamma': 0.9, 'max_depth': 6, 'labelCol': 'lab

In [55]:
best_reg

{'alpha': 0, 'lambda': 2}

### Lower the learning rate and decide the optimal parameters

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

---
## LogisticRegression Tuning

Подберем гиперпараметры для логрега из предыдущих практик

In [70]:
from pyspark.ml.classification import LogisticRegression


def objective_lr(space):
    estimator = LogisticRegression(regParam=space['regParam'], elasticNetParam=space['elasticNetParam'])
    success = False
    attempts = 0
    model = None
    while not success and attempts < 2:
        try:
            model = estimator.fit(train_df, {'lr': space['lr']})
            success = True
        except Exception as e:
            attempts += 1
            print(e)
            print('Try again')
        
    log_loss = logloss(model, val_df, probabilities_col='probability')
    roc_auc = rocauc(model, val_df, probabilities_col='probability')
    
    print('LOG-LOSS: {}, ROC-AUC: {}'.format(log_loss, roc_auc))

    return {'loss': log_loss, 'rocauc': roc_auc, 'status': STATUS_OK }

In [71]:
lr_choice = [0.0001, 0.001, 0.01, 1.]
regParam_choice = [0., .1, .4, 1., 5.]
elasticNetParam_choice = [0., .2, .5, .8, 1.]

space = {
    'lr': hp.choice('lr', lr_choice),
    'regParam': hp.choice('regParam', regParam_choice),
    'elasticNetParam': hp.choice('elasticNetParam', elasticNetParam_choice)
}

trials_lr = Trials()
best_lr = fmin(fn=objective_lr,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials_lr)

LOG-LOSS: 0.5680310722807922, ROC-AUC: 0.6998465300457369
LOG-LOSS: 0.5555410648830319, ROC-AUC: 0.7004408305645256                       
LOG-LOSS: 0.57302418343717, ROC-AUC: 0.5                                        
LOG-LOSS: 0.57302418343717, ROC-AUC: 0.5                                        
LOG-LOSS: 0.57302418343717, ROC-AUC: 0.5                                        
LOG-LOSS: 0.57302418343717, ROC-AUC: 0.5                                        
LOG-LOSS: 0.5521286398496515, ROC-AUC: 0.6823705962744437                       
LOG-LOSS: 0.57302418343717, ROC-AUC: 0.5                                        
LOG-LOSS: 0.5725193228907673, ROC-AUC: 0.6056175700978408                       
LOG-LOSS: 0.5286990365485306, ROC-AUC: 0.7035621473488969                       
LOG-LOSS: 0.57302418343717, ROC-AUC: 0.5                                         
LOG-LOSS: 0.57302418343717, ROC-AUC: 0.5                                         
LOG-LOSS: 0.5286990365485306, ROC-AUC: 0.70356214

In [72]:
best_lr

{'elasticNetParam': 4, 'lr': 3, 'regParam': 0}

---
## Optional [MongoTrials](https://hyperopt.github.io/hyperopt/scaleout/mongodb/)

> For parallel search, hyperopt includes a MongoTrials implementation that supports asynchronous updates.

**TLDR** Преимущества использования `MongoTrials`:
* `MongoTrials` позволяет параллельно запускать несколько вычислений целевой функции
* Динамический уровень параллелизма - можно добавлять/удалять воркеров, которые вычисляют целевую функцию
* Все результаты сохраняются в БД - история запусков никуда не потеряется

*За выполнение данного задания можно получить дополнительно +0.4 к итоговому баллу*

### XGBoost Tuning

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

# Results

Подведем итоги.

Обучите модели с найденными (оптимальными) гиперпараметрами и сделайте справнение на отложенной выборке

In [76]:
estimator = XGBoostEstimator(**{
    'alpha': alpha_choice[best_reg['alpha']],
    'reg_lambda': lambda_choice[best_reg['lambda']],
    'num_round': num_round_choice[best['num_round']],
    'eta': eta_choice[best['eta']],
    'max_depth': max_depth_choice[best_depth_child_weight['max_depth']],
    'min_child_weight': min_child_weight_choice[best_depth_child_weight['min_child_weight']],
    'subsample': subsample_choice[best_sample['subsample']],
    'gamma': gamma_choice[best_gamma['gamma']],
    'colsample_bytree': colsample_bytree_choice[best_sample['colsample_bytree']],   
    
    **static_params
})
model = estimator.fit(train_df)

xgb_opt_metrics = {
    'logloss': logloss(model, val_df, probabilities_col='probabilities'), 
    'rocauc': rocauc(model, val_df, probabilities_col='probabilities')
}
all_metrics['xgb_opt'] = xgb_opt_metrics

Итоговая таблица

In [77]:
get_ate(all_metrics, CONTROL_NAME)

,metric,xgb_opt ate %
0,rocauc,0.773141
1,logloss,-1.575764
